In [1]:
from generator_based_on_Seattle import *
from parameters import *

In [2]:
# generate traffic 

generate_traffic_based_on_seattle(
    from_date=FROM_DATE, 
    to_date=TO_DATE, 
    first_erlang=LAMBDA_0, 
    constant_bitrate=IS_CONSTANT_BITRATE, 
    aggregation=AGGREGATION, 
    resampling=RESAMPLING)

In [3]:
# predict three periods

perdict_traffic_for_next_period(pd.read_csv('period1.csv'))
perdict_traffic_for_next_period(pd.read_csv('period2.csv'))
perdict_traffic_for_next_period(pd.read_csv('period3.csv'))

In [4]:
# create files containing three periods, those that we have the generated ("real") and predicted traffic

generated_traffic = pd.concat([pd.read_csv('period2.csv'),pd.read_csv('period3.csv'),pd.read_csv('period4.csv')])
predicted_traffic = pd.concat([pd.read_csv('prediction_from_day_90.csv'),pd.read_csv('prediction_from_day_180.csv'),pd.read_csv('prediction_from_day_270.csv')])

In [5]:
# add column "holding time"

generated_traffic['holding_time'] = generated_traffic['departure_time'] - generated_traffic['arrival_time']
predicted_traffic['holding_time'] = predicted_traffic['departure_time'] - predicted_traffic['arrival_time']

In [6]:
# gather numbers of connections and mean holding times for s-d pairs

n_connections_generated = []
n_connections_predicted = []
n_connections_differences = []
mean_holding_time_generated = []
mean_holding_time_predicted = []
mean_holding_time_differences = []
for source_node in range(14):
    for destination_node in range(14):
        if(source_node != destination_node):
            generated_connections = generated_traffic.loc[(generated_traffic['source_id'] == source_node) & (generated_traffic['destination_id'] == destination_node)]
            predicted_connections = predicted_traffic.loc[(predicted_traffic['source_id'] == source_node) & (predicted_traffic['destination_id'] == destination_node)]
            generated_mean_holding_time = generated_connections['holding_time'].mean()
            predicted_mean_holding_time = predicted_connections['holding_time'].mean()
            if(math.isnan(generated_mean_holding_time)):
                print("{} --> {}".format(source_node,destination_node))
            if(math.isnan(predicted_mean_holding_time)):
                print("{} --> {}".format(source_node,destination_node))    

            n_connections_generated.append(len(generated_connections))
            n_connections_predicted.append(len(predicted_connections))
            n_connections_differences.append(len(predicted_connections) - len(generated_connections))
            mean_holding_time_generated.append(generated_mean_holding_time)
            mean_holding_time_predicted.append(predicted_mean_holding_time)
            mean_holding_time_differences.append(predicted_mean_holding_time-generated_mean_holding_time)


In [7]:
# print statistics regarding the number of connections per s-d pair; positive number = more predicted connections than genrated ("real") connections –> predicted traffic overestimates
# statistics contain the count, mean, std, quartiles, min and max

pd.Series(n_connections_differences).describe()   

count    182.000000
mean      39.697802
std       19.349987
min       -2.000000
25%       27.000000
50%       40.000000
75%       51.750000
max       85.000000
dtype: float64

In [8]:
# print statistics regarding the mean holding time per s-d pair; positive number = higher mean holding time in predicted connections than in the genrated ("real") connections –> predicted traffic overestimates
# statistics contain the count, mean, std, quartiles, min and max

pd.Series(mean_holding_time_differences).describe()   

count    182.000000
mean       0.334014
std        0.783938
min       -0.824488
25%       -0.210584
50%        0.137715
75%        0.734020
max        3.046193
dtype: float64